In [ ]:
# ============================================================
# Minimal DNN + Tiny Grid (X = slope + macro) — print only
# ============================================================
import os, sys, time, warnings, numpy as np, pandas as pd
warnings.filterwarnings("ignore")

from rolling_framework import Machine  # framework entrypoint

# --------------------- Paths ------------------------
DATA_DIR   = "data/"
Y_FILE     = os.path.join(DATA_DIR, "exrets.csv")
SLOPE_FILE = os.path.join(DATA_DIR, "slope.csv")
MACRO_FILE = os.path.join(DATA_DIR, "MacroFactors.csv")

# --------------------- Dates ------------------------
BURN_START, BURN_END     = "197108", "199001"
PERIOD_START, PERIOD_END = "197108", "202312"
HORIZON = 12
MATURITIES = ["xr_2","xr_3","xr_5","xr_7","xr_10"]

# --------------------- Load -------------------------
def _load_csv(path, name):
    try:
        return pd.read_csv(path, index_col="Time")
    except FileNotFoundError:
        sys.exit(f"[ERROR] Missing {name} → {path}")

def _align_time(*dfs):
    idx=None
    for d in dfs: idx = d.index if idx is None else idx.intersection(d.index)
    return [d.loc[idx].sort_index() for d in dfs]

y     = _load_csv(Y_FILE,   "exrets")
slope = _load_csv(SLOPE_FILE, "slope")
macro = _load_csv(MACRO_FILE, "macro")

y = y[[c for c in MATURITIES if c in y.columns]]
if y.empty:
    sys.exit("[ERROR] None of MATURITIES found in exrets.csv")

y, slope, macro = _align_time(y, slope, macro)
X = pd.concat([slope, macro], axis=1)

print("✓ shapes:", {"X": X.shape, "y": y.shape})
print("✓ time:", time.strftime("%Y-%m-%d %H:%M:%S"))

# --------------------- Base DNN options -------------------
opt = {
    "scaler": "standard",
    "hidden": (32, 16),
    "dropout": 0.1,
    "lr": 1e-3,
    "wd": 1e-4,   # L2 (weight decay)
    "bs": 32,
    "epochs": 150,
    "patience": 12,
    "seed": 42,
}

# --------------------- Tiny grid ----------------------
grid = {
    "dnn__module__hidden":          [(16,), (16, 8)],
    "dnn__module__dropout":         [0.2],
    "dnn__optimizer__lr":           [1e-3],
    "dnn__optimizer__weight_decay": [1e-4],
    "dnn__batch_size":              [32],
    "dnn__max_epochs":              [150],
    "dnn__patience":                [12],
}

# --------------------- Run ---------------------------
m = Machine(
    X, y, "DNN",
    option=opt, params_grid=grid,
    burn_in_start=BURN_START, burn_in_end=BURN_END,
    period=[PERIOD_START, PERIOD_END], forecast_horizon=HORIZON,
)

print("\n▶ Training start (DNN on slope+macro, tiny grid)")
start = time.time()
m.training()
elapsed = time.time() - start

# R² / MSE 계산
r2  = m.R2OOS()
mse = m.MSEOOS()

def _mean(x):
    if isinstance(x, (pd.Series, pd.DataFrame, np.ndarray, list, tuple)):
        arr = np.asarray(x, dtype=float)
        return float(np.nanmean(arr))
    try:
        return float(x)
    except Exception:
        return float("nan")

# --------------------- Print summary ---------------------
print("\n" + "="*60)
print("✅ EXPERIMENT SUMMARY")
print("="*60)
print(f"Elapsed time: {elapsed/60:.2f} min")
print("\n--- Base options ---")
for k,v in opt.items():
    print(f"{k:>10}: {v}")
print("\n--- Grid settings ---")
for k,v in grid.items():
    print(f"{k:>30}: {v}")

print("\n--- Results ---")
print("R²_OOS by maturity:")
print(r2.round(4) if isinstance(r2, pd.Series) else r2)
print("\nMSE_OOS by maturity:")
print(mse.round(4) if isinstance(mse, pd.Series) else mse)
print(f"\nR²_OOS (mean): {_mean(r2):.6f}")
print(f"MSE_OOS (mean): {_mean(mse):.6f}")

best_params = getattr(m, "best_params", None)
if best_params:
    print("\n--- Best parameters from grid ---")
    for k, v in best_params.items():
        print(f"{k:>30}: {v}")

print("\n✓ finished at", time.strftime("%Y-%m-%d %H:%M:%S"))

▶ OLS-SL_nonDNN


OLS rolling:  93%|█████████▎| 485/520 [00:08<00:00, 59.08it/s]